In [25]:
import numpy as np
import pandas as pd
import plotly
import plotly.express as px
plotly.offline.init_notebook_mode (connected = True)
import matplotlib.pyplot as plt
import warnings

In [39]:
header = 0 # 0 for No, 1 for Yes
label = 1 # 0 for No, 1 for Yes
path="iris.data.csv" #path to the datset
visualization=1 # 0 for No, 1 for Yes
apply = 1 # 0 for none, 1 for standardization, 2 for normalization
heatmap=1 # 0 for No, 1 for Yes
components = 2 # get number input from user

In [27]:
if header==1:
    df = pd.read_csv(path)
else:
    df = pd.read_csv(path,header=None)

if label==1:
    labeled=df[df.columns[-1]]
    df=df.drop([df.columns[-1]],axis=1)

In [28]:
df.head()

,0,1,2,3
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [29]:
warnings.filterwarnings("ignore")

In [30]:
if heatmap==1:
    df.corr().style.background_gradient(cmap='coolwarm') #to be worked upon for plotting by Shivesh

In [31]:
feats = df.columns[:]
feats # getting the header names

Int64Index([0, 1, 2, 3], dtype='int64')

In [32]:
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing

if apply==1:
    scaler = StandardScaler().fit_transform(df[feats])
elif apply==2:
    scaler = preprocessing.normalize(df[feats])
else:
    scaler = df[feats]

In [33]:
scaled = pd.DataFrame(scaler, columns=feats)
if label==1:
    scaled['Label'] = labeled
scaled

,0,1,2,3,Label
0,-0.900681,1.032057,-1.341272,-1.312977,Iris-setosa
1,-1.143017,-0.124958,-1.341272,-1.312977,Iris-setosa
2,-1.385353,0.337848,-1.398138,-1.312977,Iris-setosa
3,-1.506521,0.106445,-1.284407,-1.312977,Iris-setosa
4,-1.021849,1.263460,-1.341272,-1.312977,Iris-setosa
...,...,...,...,...,...
145,1.038005,-0.124958,0.819624,1.447956,Iris-virginica
146,0.553333,-1.281972,0.705893,0.922064,Iris-virginica
147,0.795669,-0.124958,0.819624,1.053537,Iris-virginica
148,0.432165,0.800654,0.933356,1.447956,Iris-virginica


In [34]:
#before applying PCA Visualization #Shivesh add this as title
if label==1 and visualization==1:
    features = feats
    fig = px.scatter_matrix(
        scaled,
        dimensions=features,
        color="Label"
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()

In [35]:
from sklearn.decomposition import PCA
def create_pca(X, y, n_components=None):
    if (n_components != None):
        pca = PCA(n_components)
    else:
        pca = PCA()
    trans = pca.fit_transform(X)
    trans = pd.DataFrame(trans)
    trans['species'] = y
    return pca, trans

In [41]:
if label==1:
    X, y = scaled.drop(['Label'], axis=1), scaled['Label']

    if len(feats)>components:
        pca1, trans1 = create_pca(X=X, y=y, n_components=components)
        print(pca1.explained_variance_ratio_)#mention what it is : Shivesh
        print(pca1.components_)# mention this to!

    

[0.72770452 0.23030523]
[[ 0.52237162 -0.26335492  0.58125401  0.56561105]
 [ 0.37231836  0.92555649  0.02109478  0.06541577]]


In [43]:
if label==1:

    pca = PCA()
    components = pca.fit_transform(df[features])
    labels = {
        str(i): f"PC {i+1} ({var:.1f}%)"
        for i, var in enumerate(pca.explained_variance_ratio_ * 100)
    }
    fig = px.scatter_matrix(
    components,
    labels=labels,
    dimensions=range(len(features)),
    color=scaled["Label"]
    )
    fig.update_traces(diagonal_visible=True)
    fig.show()